In [1]:
import collections
import random
import gym
import numpy as np

In [2]:
from cartPoleDqn import DQN

In [3]:
class Agent:
    def __init__(self):
        pass

    def get_action(self):
        pass

    def train(self):
        pass

    def remember(self):
        pass
    
    def replay(self):
        pass

    def play(self):
        pass